In [66]:
%%capture 
!pip install -r requirements.txt

In [67]:
from enum import Enum, StrEnum

class Scaling(Enum):
    INDEPENDENT = 1
    JOINT = 2

class DatasetType(StrEnum):
    TRAIN = 'trn'
    TEST = 'tst'
    VALIDATION = 'trn'

# Global variable to enable debug mode
DEBUG = True

#### Data loading & pre-processing functions

In [68]:
import pandas as pd
import numpy as np
from sklearn import preprocessing


# *Helper function to preprocess the RSSI data
def preprocess_rssi_data(df_rssi: pd.DataFrame, scaling_strategy: Scaling) -> pd.DataFrame:
    """
    This function preprocesses the training data by:
    1. Replacing all 100 values with -110 (ensures continuity of data)
    2. Separating the RSS values from the labels
    3. Scaling the data to have zero mean and unit variance

    Parameters:
    - train: The training data to be preprocessed
    - scaling_strategy: The scaling strategy to be used (INDEPENDENT or JOINT)
    """
    
    # 1. replace all 100 values with -110 (ensures continuity of data)
    df = df_rssi.replace(100, -110)
    
    # 2. Separate the RSS values from the labels
    rssiValues = df.iloc[:, :-3]
    labels = df.iloc[:, -3:]
    
    # 3. Scale the data to have zero mean and unit variance
    # This is done either independently for each AP or jointly for all APs
    if scaling_strategy == Scaling.INDEPENDENT:
        scaler = preprocessing.StandardScaler()

        scaled_rss = scaler.fit_transform(rssiValues)
        df_scaled_rss = pd.DataFrame(scaled_rss, columns=rssiValues.columns)
        df = pd.concat([df_scaled_rss, labels], axis=1)
    
    elif scaling_strategy == Scaling.JOINT:
        flattened = rssiValues.values.flatten()
        global_mean = np.mean(flattened)
        global_std = np.std(flattened)
        
        scaled_rss = (rssiValues - global_mean) / global_std
        df = pd.concat([scaled_rss, labels], axis=1)
        df = df.reset_index(drop=True)
    
    else: 
        raise NotImplementedError("Specified scaling strategy is not implemented, use either Scaling.INDEPENDENT or Scaling.JOINT.")
    
    return df

# # *Load and pre-process the training data
# def get_preprocessed_training_data(data_path: str, training_months: list[str], num_APs: int, scaling_strategy: Scaling, floor: int) -> pd.DataFrame:
#     """
#     This function loads and preprocesses the training data from the specified training months and floor.

#     Parameters:
#     - data_path: The path to the data
#     - training_months: The list of training months to be used
#     - num_APs: The number of access points
#     - scaling_strategy: The scaling strategy to be used (INDEPENDENT or JOINT)
#     - floor: The floor to be used
#     """
#     # Since the csv files do not have column names, we define these first.
#     list_of_APs = ["AP" + str(i) for i in range(0, num_APs)]

#     # Load the training data from all specified training sets.  
#     df_rss = pd.concat([pd.read_csv(data_path + training_set + 'trn01rss.csv', names=list_of_APs) for training_set in training_months])
#     df_rss = df_rss.reset_index(drop=True)
    
#     # Get all x,y,floor labels (gotten from data_path + training_month + 'trn01crd.csv')
#     df_labels = pd.concat([pd.read_csv(data_path + training_set + 'trn01crd.csv', names=['x', 'y', 'floor']) for training_set in training_months])
#     df_labels = df_labels.reset_index(drop=True)

#     # Add the labels to the pre-processed data
#     df_labeled = pd.concat([df_rss, df_labels], axis=1)
    
#     # Filter the data to only include the specified floor
#     df_labeled = df_labeled[df_labeled['floor'] == floor]

#     # Pre-processing of the training data
#     df_train = preprocess_rssi_data(df_labeled, scaling_strategy)
    
#     return df_train

# *Load and pre-process the data
def get_preprocessed_dataset(data_path: str, months: list[str], sets: list[str], type: DatasetType, num_APs: int, scaling_strategy: Scaling, floor: int) -> pd.DataFrame:
    """
    This function loads and preprocesses the training data from the specified training months and floor.

    Parameters:
    - data_path: The path to the data
    - months: The list of months to be used
    - sets: The list of set numbers to be used
    - type: The type of dataset to be made (TRAIN, TEST or VALIDATION)
    - num_APs: The number of access points
    - scaling_strategy: The scaling strategy to be used (INDEPENDENT or JOINT)
    - floor: The floor to be used
    """
    # Since the csv files do not have column names, we define these first.
    list_of_APs = ["AP" + str(i) for i in range(0, num_APs)]

    # Load the test data from all specified test sets.  
    df_test_rss = pd.concat([pd.read_csv(data_path + month + '/' + type + set + 'rss.csv', names=list_of_APs) for month in months for set in sets])
    df_test_rss = df_test_rss.reset_index(drop=True)
    
    # Get all x,y,floor labels
    df_test_labels = pd.concat([pd.read_csv(data_path + month + '/' + type + set + 'crd.csv', names=['x', 'y', 'floor']) for month in months for set in sets])
    df_test_labels = df_test_labels.reset_index(drop=True)

    # Add the labels to the pre-processed data
    df_test_labeled = pd.concat([df_test_rss, df_test_labels], axis=1)
    
    # Filter the data to only include the specified floor
    df_test_labeled = df_test_labeled[df_test_labeled['floor'] == floor]

    # Pre-processing of the training data
    df_test = preprocess_rssi_data(df_test_labeled, scaling_strategy)
    
    return df_test
    

#### SETUP

In [69]:
data_path = './data/'
training_months = ['01', '02', '03', '04', '05']
sets = ['01']
type = DatasetType.TRAIN
num_APs = 620
scaling_strategy = Scaling.JOINT
floor = 3


df_train_full = get_preprocessed_dataset(data_path, training_months, sets, type, num_APs, scaling_strategy, floor)
df_train_x = df_train_full.iloc[:, :-3] # Just the RSSI values
df_train_y = df_train_full.iloc[:, -3:-1] # Just the x and y coordinates (no floor)

if DEBUG: print('df_train_full:', df_train_full.shape)


df_train_full: (1440, 623)


In [70]:
months = ['01', '02', '03', '04', '05']
sets = ['01'] # 01 Corresponds to the same locations as the training set
type = DatasetType.TEST

df_test_full = get_preprocessed_dataset(data_path, months, sets, type, num_APs, scaling_strategy, floor)
df_test_x = df_test_full.iloc[:, :-3] # Just the RSSI values
df_test_y = df_test_full.iloc[:, -3:-1] # Just the x and y coordinates (no floor)

if DEBUG: print('df_test_full:', df_test_full.shape)

df_test_full: (1440, 623)


In [71]:
months = ['01']
sets = ['02', '03', '04']
type = DatasetType.VALIDATION

df_val_full = get_preprocessed_dataset(data_path, months, sets, type, num_APs, scaling_strategy, floor)
df_val_x = df_val_full.iloc[:, :-3] # Just the RSSI values
df_val_y = df_val_full.iloc[:, -3:-1] # Just the x and y coordinates (no floor)

if DEBUG: print('df_val_full:', df_val_full.shape)

df_val_full: (864, 623)


In [72]:
import torch
from torch.utils.data import DataLoader, TensorDataset

# Since the implementations will be made in PyTorch, we convert the data to PyTorch tensors
X_train_tensor = torch.tensor(df_train_x.values, dtype=torch.float32)
y_train_tensor = torch.tensor(df_train_y.values, dtype=torch.float32)
X_test_tensor = torch.tensor(df_test_x.values, dtype=torch.float32)
y_test_tensor = torch.tensor(df_test_y.values, dtype=torch.float32)
X_val_tensor = torch.tensor(df_val_x.values, dtype=torch.float32)
y_val_tensor = torch.tensor(df_val_y.values, dtype=torch.float32)

# Get the data via DataLoaders
t_training = TensorDataset(X_train_tensor, y_train_tensor)
t_test = TensorDataset(X_test_tensor, y_test_tensor)
t_val = TensorDataset(X_val_tensor, y_val_tensor)

# train_loader = DataLoader(t_training, batch_size=16, shuffle=True)
# test_loader = DataLoader(t_test, batch_size=16, shuffle=True)
# val_loader = DataLoader(t_val, batch_size=16, shuffle=True)



# Full-Input Networks
These networks take the full input of 620 features to perform x,y predictions.

#### MultiLayer Perceptrons

In [73]:
SEARCH_MLP_FULL = True

In [74]:
import torch.nn as nn

class MLPfull(nn.Module):
    def __init__(self, hidden_layer_sizes, dropout_rate):
        super(MLPfull, self).__init__()
        
        layers = []
        input_dim = 620
        
        # Make it easier to grid-search different sizes of hidden layers
        for hidden_dim in hidden_layer_sizes:
            layers.append(nn.Linear(input_dim, hidden_dim))
            layers.append(nn.ReLU())
            layers.append(nn.Dropout(dropout_rate))
            input_dim = hidden_dim # Ensure all hidden layers are constant size
        
        # At this point we know input_dim equals the output size of the last hidden layer, so we can re-use it here.
        layers.append(nn.Linear(input_dim, 2)) # x,y output
        
        # Construct the actual model based on the layers defined above.
        self.model = nn.Sequential(*layers)
        
    def forward(self, x):
        return self.model(x)

In [75]:
def train_MLP_full(model, train_loader, val_loader, criterion, optimizer, epochs):
    for epoch in range(epochs):
        model.train() # Sets the model to training mode
        running_loss = 0.0 # Keep track of the (MSE) loss
        
        # Actual training loop
        for inputs, labels in train_loader:
            optimizer.zero_grad() # Reset gradients from last iteration
            outputs = model(inputs) # Forward pass
            loss = criterion(outputs, labels) # Compute the loss (MSE) between the predictions and the ground-truth labels
            loss.backward() # Perform backpropagation
            optimizer.step() # Update model parameters (weights) based on the gradients computed during backpropagation
            running_loss += loss.item() # Running loss is the sum of the losses for all batches FOR THE CURRENT EPOCH <-- TODO: (Make list for final model to plot)
        
        # Validation time
        model.eval()
        val_loss = 0.0 # Accumulated validation loss
        
        # Validation loop
        with torch.no_grad(): # No need to compute gradients during validation
            for inputs, labels in val_loader:
                outputs = model(inputs) # Forward pass to get predictions
                loss = criterion(outputs, labels) # Compute the loss (MSE) between the predictions and the ground-truth labels
                val_loss += loss.item() # Accumulate the validation loss for this epoch <-- TODO: (Make list for final model to plot)
        
        # Print the loss for this epoch
        print(f'Epoch {epoch+1}/{epochs} - Avg Training Loss: {running_loss/len(train_loader)} - Avg Validation Loss: {val_loss/len(val_loader)}')
    
    print('Finished Training')
    return val_loss/len(val_loader) # Return the average validation loss for final epoch

In [76]:
import optuna

def MLP_full_gridsearch(trial):
    # Hyper-parameters to be optimized
    
    # The line below does not work due to a optuna limitation. It is kept here for reference.
    #! hidden_layer_sizes = trial.suggest_categorical('hidden_layer_sizes', [ (v,) * i for v in [700, 512, 256, 128] for i in range(2, 5)])
    
    hidden_layer_size = trial.suggest_categorical('hidden_layer_size', [700, 512, 256, 128])
    hidden_layer_count = trial.suggest_int('hidden_layer_count', 2, 4) # inclusive
    hidden_layer_sizes = (hidden_layer_size,) * hidden_layer_count
    
    dropout_rate = trial.suggest_float('dropout_rate', 0.4, 0.6)
    lr = trial.suggest_float('lr', 0.001, 0.01)
    batch_size = trial.suggest_int('batch_size', 16, 512, step=16)
    epochs = trial.suggest_int('epochs', 50, 150)
    
    # Initialize the model
    model = MLPfull(hidden_layer_sizes, dropout_rate)
    optimizer = torch.optim.Adam(model.parameters(), lr=lr)
    criterion = nn.MSELoss()
    
    # Use chosen batch size instead of pre-defined one
    train_loader = DataLoader(t_training, batch_size=batch_size, shuffle=True)
    val_loader = DataLoader(t_val, batch_size=batch_size, shuffle=True)
    
    # Train the model, return validation loss
    val_loss = train_MLP_full(model, train_loader, val_loader, criterion, optimizer, epochs)
    
    return val_loss

if SEARCH_MLP_FULL:
    print('Starting MLP full grid search')

    # Optuna study object and direction (minimize validation loss)
    study = optuna.create_study(direction='minimize')
    study.optimize(MLP_full_gridsearch, n_trials=25)

    print('====================================')
    print('Number of finished trials:', len(study.trials))
    print('Best trial:')
    trial = study.best_trial
    print('     Value: ', trial.value)
    print('     Params: ')
    for key, value in trial.params.items():
        print(f'         {key}: {value}')

else: print('Skipping MLP full grid search')

[I 2024-06-05 23:41:47,268] A new study created in memory with name: no-name-6e049453-b9da-4b2d-9c10-c61f598e5d3d


Starting MLP full grid search
Epoch 1/116 - Avg Training Loss: 81.87222771022631 - Avg Validation Loss: 45.491198675973074


C:\Users\Robbie\AppData\Local\Temp\ipykernel_16892\2979055314.py:15: FutureWarning: suggest_int() got {'step'} as positional arguments but they were expected to be given as keyword arguments.
  batch_size = trial.suggest_int('batch_size', 16, 512, 16)


Epoch 2/116 - Avg Training Loss: 18.216088916944422 - Avg Validation Loss: 13.789060320172991
Epoch 3/116 - Avg Training Loss: 11.358692666758662 - Avg Validation Loss: 7.885433673858643
Epoch 4/116 - Avg Training Loss: 9.182460183682648 - Avg Validation Loss: 5.693149941308158
Epoch 5/116 - Avg Training Loss: 8.182089826335078 - Avg Validation Loss: 5.450273547853742
Epoch 6/116 - Avg Training Loss: 7.421233612558116 - Avg Validation Loss: 5.083327719143459
Epoch 7/116 - Avg Training Loss: 7.105399691540262 - Avg Validation Loss: 5.130536249705723
Epoch 8/116 - Avg Training Loss: 6.421888641689135 - Avg Validation Loss: 5.021373936108181
Epoch 9/116 - Avg Training Loss: 6.89305301334547 - Avg Validation Loss: 4.27212883744921
Epoch 10/116 - Avg Training Loss: 6.190495242243228 - Avg Validation Loss: 6.589968034199306
Epoch 11/116 - Avg Training Loss: 6.045168296150539 - Avg Validation Loss: 11.97100237437657
Epoch 12/116 - Avg Training Loss: 6.564402372940727 - Avg Validation Loss: 7.

[W 2024-06-05 23:41:53,369] Trial 0 failed with parameters: {'hidden_layer_size': 700, 'hidden_layer_count': 3, 'dropout_rate': 0.44167346306117994, 'lr': 0.0012904030281354873, 'batch_size': 64, 'epochs': 116} because of the following error: KeyboardInterrupt().
Traceback (most recent call last):
  File "c:\Users\Robbie\AppData\Local\Programs\Python\Python311\Lib\site-packages\optuna\study\_optimize.py", line 196, in _run_trial
    value_or_values = func(trial)
                      ^^^^^^^^^^^
  File "C:\Users\Robbie\AppData\Local\Temp\ipykernel_16892\2979055314.py", line 28, in MLP_full_gridsearch
    val_loss = train_MLP_full(model, train_loader, val_loader, criterion, optimizer, epochs)
               ^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^
  File "C:\Users\Robbie\AppData\Local\Temp\ipykernel_16892\1111882985.py", line 9, in train_MLP_full
    outputs = model(inputs) # Forward pass
              ^^^^^^^^^^^^^
  File "c:\Users\Robbie\AppData\Lo

KeyboardInterrupt: 